In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams['figure.figsize']=(20,5)
import plotly.express as px

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import pyarrow.parquet as pq

In [4]:
df=pd.read_parquet("d://para_All_28_id.parquet")

In [6]:
df=pd.read_parquet("d:/complete.parquet")


In [10]:
le = LabelEncoder()
df['label'] = le.fit_transform(df["sensor"])

In [17]:
df

,sensor,Clock,R_Voltage,Y_Voltage,B_Voltage,R_Current,Y_Current,B_Current,Kwh,label
0,5f718b613291c7.03696209,2022-11-18 00:30:00,6579.0,6616.0,6631.0,3.18,3.34,4.88,8800.0,0
1,5f718b613291c7.03696209,2022-11-18 01:30:00,6605.0,6642.0,6655.0,3.36,3.41,4.97,8200.0,0
2,5f718b613291c7.03696209,2022-11-18 02:00:00,6644.0,6681.0,6694.0,3.40,3.58,5.18,8300.0,0
3,5f718b613291c7.03696209,2022-11-18 02:30:00,6656.0,6694.0,6707.0,3.42,3.57,5.22,8000.0,0
4,5f718b613291c7.03696209,2022-11-18 03:00:00,6656.0,6696.0,6708.0,3.39,3.58,5.22,7800.0,0
...,...,...,...,...,...,...,...,...,...,...
473710,641c17bc672215.97177522,2023-10-31 23:30:00,6535.0,6485.0,6509.0,23.45,25.06,24.47,236100.0,27
473711,641c17bc672215.97177522,2023-10-31 21:30:00,6555.0,6501.0,6531.0,24.22,24.58,22.41,232000.0,27
473712,641c17bc672215.97177522,2023-11-01 00:00:00,6577.0,6524.0,6550.0,21.91,24.01,23.65,226200.0,27
473713,641c17bc672215.97177522,2023-11-01 00:30:00,6609.0,6554.0,6581.0,18.70,20.94,20.09,196000.0,27


In [41]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [39]:
df11=df.groupby('sensor')

In [40]:
for i ,y in df11:
    print("i",type(i))
    print("y",type(y))
    break

i <class 'str'>
y <class 'pandas.core.frame.DataFrame'>


In [38]:
d={}
for i , y in df11:
    sensor_df=y
    sensor_df['Clock'] = pd.to_datetime(sensor_df['Clock'])
    sensor_df.set_index(['Clock'], inplace=True, drop=True)
    sensor_df = sensor_df[sensor_df.index >= '2022-11-18 00:00:00']
    print(sensor_df.head())
    # dfresample['label'] = i
    # dfresample = sensor_df[['Kwh']].resample(rule='1H').sum()
    # dfresample['label']=i
    dfresample_label = sensor_df[['Kwh']].resample(rule='1H').sum()
    dfresample_label['label'] = sensor_df['label']  # Assign the label
    # dfresample = pd.concat([dfresample, dfresample_label])
    # print(dfresample_label.head(1))
    d[f'key_{i}'] = dfresample_label.head(1)


                                      sensor  R_Voltage  Y_Voltage  B_Voltage  \
Clock                                                                           
2022-11-18 00:30:00  5f718b613291c7.03696209     6579.0     6616.0     6631.0   
2022-11-18 01:30:00  5f718b613291c7.03696209     6605.0     6642.0     6655.0   
2022-11-18 02:00:00  5f718b613291c7.03696209     6644.0     6681.0     6694.0   
2022-11-18 02:30:00  5f718b613291c7.03696209     6656.0     6694.0     6707.0   
2022-11-18 03:00:00  5f718b613291c7.03696209     6656.0     6696.0     6708.0   

                     R_Current  Y_Current  B_Current     Kwh  label  
Clock                                                                
2022-11-18 00:30:00       3.18       3.34       4.88  8800.0      0  
2022-11-18 01:30:00       3.36       3.41       4.97  8200.0      0  
2022-11-18 02:00:00       3.40       3.58       5.18  8300.0      0  
2022-11-18 02:30:00       3.42       3.57       5.22  8000.0      0  
2022-11-18 0

ValueError: cannot reindex on an axis with duplicate labels

In [37]:
# d

In [26]:
dfresample_label

,Kwh,label
Clock,,
2023-02-04 00:00:00,118100.0,27
2023-02-04 01:00:00,203500.0,27
2023-02-04 02:00:00,181500.0,27
2023-02-04 03:00:00,172000.0,27
2023-02-04 04:00:00,189400.0,27
...,...,...
2023-10-31 21:00:00,480500.0,27
2023-10-31 22:00:00,457900.0,27
2023-10-31 23:00:00,452200.0,27


In [27]:
dfresample_label['sensor'] = le.inverse_transform(dfresample_label['label'])

In [28]:
dfresample_label

,Kwh,label,sensor
Clock,,,
2023-02-04 00:00:00,118100.0,27,641c17bc672215.97177522
2023-02-04 01:00:00,203500.0,27,641c17bc672215.97177522
2023-02-04 02:00:00,181500.0,27,641c17bc672215.97177522
2023-02-04 03:00:00,172000.0,27,641c17bc672215.97177522
2023-02-04 04:00:00,189400.0,27,641c17bc672215.97177522
...,...,...,...
2023-10-31 21:00:00,480500.0,27,641c17bc672215.97177522
2023-10-31 22:00:00,457900.0,27,641c17bc672215.97177522
2023-10-31 23:00:00,452200.0,27,641c17bc672215.97177522


In [21]:

dfresample['sensor'] = le.inverse_transform(dfresample['label'])
# dfresample['sensor'] = le.inverse_transform(df['label'])

KeyError: 'label'

In [ ]:

# decoded_labels = label_encoder.inverse_transform(encoded_labels)

# print("Decoded Labels:", decoded_labels)